In [4]:
import http.client
import pandas as pd
import json
pd.set_option("display.max_rows", 200)

In [5]:
import http.client
conn = http.client.HTTPConnection("api.sportradar.us")
conn.request("GET", "/golf-t2/leaderboard/pga/2018/tournaments/b404a8d5-5e33-4417-ae20-5d4d147042ee/leaderboard.json?api_key=u2rg6jhkezj6dkutjcmfqc8e")
res = conn.getresponse()
data = res.read()
d = data.decode("utf-8")

In [6]:
lb_df = pd.DataFrame(json.loads(d)['leaderboard'])
lb_df["full_name"] = lb_df["first_name"].str.lower().str.replace(" ", "") + lb_df["last_name"].str.lower().str.replace(" ", "")
lb_df

,country,first_name,id,last_name,money,points,position,rounds,score,strokes,tied,full_name
0,UNITED STATES,Jordan,3e4963cb-6e80-4393-85cf-2aecec453c4a,Spieth,None,None,1,"[{'score': -6, 'strokes': 66, 'thru': 18, 'eag...",-6,66,False,jordanspieth
1,UNITED STATES,Matt,047de4e6-1cfe-4166-90e8-492add1161e3,Kuchar,None,None,2,"[{'score': -4, 'strokes': 68, 'thru': 18, 'eag...",-4,68,True,mattkuchar
2,UNITED STATES,Tony,f33d746b-8e14-4325-a4e1-81d6f983b163,Finau,None,None,2,"[{'score': -4, 'strokes': 68, 'thru': 18, 'eag...",-4,68,True,tonyfinau
3,SWEDEN,Henrik,24c9d97d-1801-459e-9778-c37ad6d20ecf,Stenson,None,None,4,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eag...",-3,69,True,henrikstenson
4,SPAIN,Rafael,12e24cec-7dbe-45d7-ab6a-f8a6de24740f,Cabrera-Bello,None,None,4,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eag...",-3,69,True,rafaelcabrera-bello
5,UNITED STATES,Patrick,12195ed9-3fcd-486a-86b4-1ec726d201cd,Reed,None,None,4,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eag...",-3,69,True,patrickreed
6,CANADA,Adam,085c0b40-e456-4d46-b011-f992a4ee8a68,Hadwin,None,None,4,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eag...",-3,69,True,adamhadwin
7,NORTHERN IRELAND,Rory,da226913-b804-48de-adbf-96e956eb75ac,McIlroy,None,None,4,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eag...",-3,69,True,rorymcilroy
8,UNITED STATES,Charley,25366155-7268-4630-baf7-7a27d5e18385,Hoffman,None,None,4,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eag...",-3,69,True,charleyhoffman
9,CHINA,Hao-Tong,81a12293-b97e-4b11-9686-a1e0bc8a4096,Li,None,None,4,"[{'score': -3, 'strokes': 69, 'thru': 18, 'eag...",-3,69,True,hao-tongli


In [7]:
df = pd.read_csv("masters.csv")
df = df[["Pool Player", "Player 1", "Player 2", "Player 3 ", "Player 4", 
         "Player 5", "Player 6", "Player 7", "Total", "Tie Breaker"]]

In [8]:
def get_score(player):
    player = player.lower().replace(" ", "")
    strokes = lb_df.loc[lb_df["full_name"] == player]["strokes"]
    return int(strokes)

In [9]:
for index, row in df.iterrows():
    score_accum = 0
    for player in row.values[1:7]:
        score_accum += get_score(player)
    df.set_value(index, "Total", score_accum)

In [10]:
df

,Pool Player,Player 1,Player 2,Player 3,Player 4,Player 5,Player 6,Player 7,Total,Tie Breaker
0,Alexander Clausen,Justin Thomas,Matt Kuchar,Gary Woodland,Kevin Chappell,Thomas Pieters,Tiger Woods,Russell Henley,443.0,277.0
1,Chris Stine,Rory McIlroy,Phil Mickelson,Brian Harman,Louis Oosthuizen,Cameron Smith,Tiger Woods,Ryan Moore,427.0,280.0
2,Dave Peterson,Rory McIlroy,Jason Day,Patrick Reed,Tony Finau,Jason Dufner,Tiger Woods,Martin Kaymer,427.0,278.0
3,Dave Powell,Dustin Johnson,Phil Mickelson,Bubba Watson,Louis Oosthuizen,Thomas Pieters,Martin Kaymer,Adam Scott,434.0,278.0
4,David Brown,Dustin Johnson,Jason Day,Bubba Watson,Patrick Cantlay,Thomas Pieters,Tiger Woods,Russell Henley,442.0,279.0
5,Don Martin,Jordan Spieth,Phil Mickelson,Kevin Kisner,Louis Oosthuizen,Kyle Stanley,Tiger Woods,Bryson DeChambeau,424.0,279.0
6,Jason Grill,Dustin Johnson,Jason Day,Bubba Watson,Tony Finau,Thomas Pieters,Tiger Woods,Adam Scott,435.0,280.0
7,Jeremy Szyba,rickie Fowler,Paul Casey,Bubba Watson,Louis Oosthuizen,Jason Dufner,Jimmy Walker,Zack Johnson,434.0,277.0
8,Jeremy Szyba #2,Rory McIlroy,Pat Perez,Charley Hoffman,Brendan Steele,Jason Dufner,Tiger Woods,Adam Scott,433.0,274.0
9,Jeremy Szyba #3,Dustin Johnson,Phil Mickelson,Patrick Reed,Branden Grace,Adam Hadwin,Daniel Berger,Ian Poulter,427.0,278.0


In [11]:
df.to_csv("masters_results.csv")